**Visual Graph Learning with GCN for Financial Forecasting: CNN-Based
OHLC Image Embeddings and Market Topology**

# **CODE 2 : CNN_Test**

# 🛠️ Setup

In [ ]:
# @title Import packages

import torch
import torch.nn as nn
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn.functional as F
import pickle as pkl
from collections import OrderedDict
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from copy import deepcopy
from tqdm import tqdm
from PIL import Image
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset, DataLoader

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    confusion_matrix,
    roc_curve,
    auc
)
import seaborn as sns

In [ ]:
# @title Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# CNN 模型定義

In [ ]:
# @title 20-day CNN Architecture
# (BasicBlock, Bottleneck, CNN20d)

class BasicBlock(nn.Module):
    """
    ResNet 基礎殘差區塊 (對應圖中的 5x3 -> 5x3)
    """
    def __init__(self, in_channels, out_channels):
        super(BasicBlock, self).__init__()

        # --- 主路徑 (Main Path) ---
        self.main_path = nn.Sequential(
            # 1. 5x3 Conv
            nn.BatchNorm2d(in_channels),
            nn.GELU(),
            nn.Conv2d(in_channels, out_channels, kernel_size=(5, 3), stride=1, padding=(2, 1)),
            # 2. 5x3 Conv
            nn.BatchNorm2d(out_channels),
            nn.GELU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=(5, 3), stride=1, padding=(2, 1)),

        )

        # --- 跳接 (Shortcut Path) ---
        self.shortcut = nn.Sequential() # 默認為 Identity

        # 如果輸入/輸出通道數不同 (例如 64 -> 128)，
        # 我們需要一個 1x1 卷積來匹配維度
        if in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        main_out = self.main_path(x)
        shortcut_out = self.shortcut(x)
        out = main_out + shortcut_out # 不加 final activation
        return out

class BottleneckBlock(nn.Module):
    """
    ResNet 瓶頸殘差區塊 (對應圖中的 1x1 -> 5x3 -> 1x1)
    """
    def __init__(self, in_channels, out_channels, squeeze_channels):
        """
        Args:
            in_channels (int): 輸入通道數
            out_channels (int): 輸出通道數 (例如 512)
            squeeze_channels (int): 瓶頸壓縮後的通道數 (例如 128)
        """
        super(BottleneckBlock, self).__init__()

        # --- 主路徑 (Main Path) ---
        self.main_path = nn.Sequential(
            # 1. 1x1 Conv (壓縮)
            nn.BatchNorm2d(in_channels),
            nn.GELU(),
            nn.Conv2d(in_channels, squeeze_channels, kernel_size=1, stride=1, padding=0),


            # 2. 5x3 Conv (您的核心卷積)
            nn.BatchNorm2d(squeeze_channels),
            nn.GELU(),
            nn.Conv2d(squeeze_channels, squeeze_channels, kernel_size=(5, 3), stride=1, padding=(2, 1)),


            # 3. 1x1 Conv (擴展)
            nn.BatchNorm2d(squeeze_channels),
            nn.GELU(),
            nn.Conv2d(squeeze_channels, out_channels, kernel_size=1, stride=1, padding=0),
        )

        # --- 跳接 (Shortcut Path) ---
        self.shortcut = nn.Sequential() # 默認為 Identity

        # 如果輸入/輸出通道數不同 (例如 256 -> 512)，
        # 或這是第一個區塊，我們需要一個 1x1 卷積來匹配維度
        if in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        main_out = self.main_path(x)
        shortcut_out = self.shortcut(x)
        out = main_out + shortcut_out # 不加 final activation
        return out

class CNN20d(nn.Module):

    def init_weights(self, m):
        if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
            torch.nn.init.xavier_uniform_(m.weight)
            if m.bias is not None:
                m.bias.data.fill_(0.01)

    def _make_layer(self, block_type, in_channels, out_channels, num_blocks, squeeze_channels=None):
        """
        建立一個包含 N 個殘差區塊的層 (Stage)
        """
        layers = []

        # 第一個區塊：處理輸入/輸出通道數不同的情況
        if block_type == BottleneckBlock:
            layers.append(block_type(in_channels, out_channels, squeeze_channels))
        else: # BasicBlock
            layers.append(block_type(in_channels, out_channels))

        # 剩餘的 (num_blocks - 1) 個區塊
        for _ in range(1, num_blocks):
            if block_type == BottleneckBlock:
                # 後續區塊的 in_channels 等於 out_channels
                layers.append(block_type(out_channels, out_channels, squeeze_channels))
            else: # BasicBlock
                layers.append(block_type(out_channels, out_channels))

        return nn.Sequential(*layers)

    def __init__(self, num_classes=2, regression_outputs=1):
        super(CNN20d, self).__init__()

        # 1. Stem (莖幹層) - 根據您的 H/W 調整
        self.conv1 = nn.Sequential(
            # 使用 5x3 卷積核 (kernel_size=(5, 3)) 更適合您的非方形輸入
            # padding=(2, 1) 可以在 stride=1 時保持 H 和 W 尺寸不變
            nn.BatchNorm2d(1),
            nn.GELU(),
            nn.Conv2d(1, 64, kernel_size=(5, 3), stride=1, padding=(2, 1))
        )

        self.pool1 = nn.MaxPool2d((2, 1)) # 第一個 MaxPool

        # 2. Stage 1 - 對應圖中 x2 (BasicBlock, 64)
        self.stage1 = self._make_layer(BasicBlock, 64, 64, num_blocks=2)

        # 3. Stage 2 - 對應圖中 x2 (BasicBlock, 128)
        self.stage2 = self._make_layer(BasicBlock, 64, 128, num_blocks=2)

        # 4. Stage 3 - 對應圖中 x2 (BasicBlock, 256)
        self.stage3 = self._make_layer(BottleneckBlock, 128, 256, num_blocks=2, squeeze_channels=64)
        self.pool2 = nn.MaxPool2d((2, 1)) # 第二個 MaxPool

        # 5. Stage 4 - 對應圖中 x2 (Bottleneck, 128 -> 512)
        # 瓶頸壓縮通道數為 128，輸出通道數為 512
        self.stage4 = self._make_layer(BottleneckBlock, 256, 512, num_blocks=2, squeeze_channels=128)


        # 6. Final Pooling (全局平均池化)
        # 這會將 (Batch, 512, H, W) -> (Batch, 512, 1, 1)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        # 7. Dropout 和全連接層
        self.DropOut = nn.Dropout(p=0.5)

        # flattened_size 現在固定為最後的通道數 512
        self.classification_fc = nn.Linear(512, num_classes)
        self.classification_activation = nn.Softmax(dim=1)

        # self.regression_fc = nn.Linear(512, regression_outputs)
        # self.regression_activation = nn.Identity()

        self.apply(self.init_weights)

    def forward(self, x, task_type='classification', embedding=None, return_params=False):
        # 確保輸入是 (Batch, Channels, H, W)
        # 原始碼是 (Batch, H, W)，所以 unsqueeze(1) 變成 (Batch, 1, H, W)
        x = x.unsqueeze(1).to(torch.float32)

        outputs = {}

        # 按照架構圖順序執行
        x = self.conv1(x)
        x = self.pool1(x)
        outputs['conv1'] = x

        x = self.stage1(x)
        outputs['stage1'] = x

        x = self.stage2(x)
        outputs['stage2'] = x

        x = self.stage3(x)
        x = self.pool2(x)
        outputs['stage3'] = x

        x = self.stage4(x)
        outputs['stage4'] = x

        x = self.avgpool(x)
        # x shape: (Batch, 512, 1, 1)

        # 展平 (Flatten)
        x = torch.flatten(x, 1) # (Batch, 512)

        x = self.DropOut(x)
        outputs['flatten'] = x

        if embedding in outputs:
            return outputs[embedding]

        # Task-specific output (這部分邏輯與您原碼相同)
        if task_type == 'classification':
            x = self.classification_fc(x)
            x = self.classification_activation(x)
        # elif task_type == 'regression':
        #     x = self.regression_fc(x)
        #     x = self.regression_activation(x)

        # (這部分邏輯與您原碼相同)
        if return_params and embedding:
            return {
                'params': dict(self.named_parameters()),
                'output': outputs[embedding]
            }

        return x

# 輔助函式

In [ ]:
def make_y_labels(y, mode="top_k", top_k=9, threshold=0.003):
    """
    根據 'mode' 參數決定 label 的生成方式：
    - mode="top_k":    (預設) 依據同一天的報酬率排名，標出 top_k 為 1，其餘為 0。
    - mode="threshold": 依據 'Return_5D' >= threshold (預設 0.003) 為 1，其餘為 0。
    """
    y = y.copy()
    y.columns = ['TradeDate', 'Ticker', 'Return_5D']

    if mode == "top_k":   # top_k=9
        y['rank'] = y.groupby('TradeDate')['Return_5D'].rank(ascending=False, method='first')
        y['label'] = (y['rank'] <= top_k).astype(int)
    elif mode == "threshold":   # threshold=0.003
        y['label'] = (y['Return_5D'] >= threshold).astype(int)
    else:
        raise ValueError(f"未知的 mode: '{mode}'")

    # 轉換成 tensor
    y_tensor = torch.tensor(y['label'].values, dtype=torch.long)
    dataset = TensorDataset(y_tensor)

    return dataset, y # 多回傳一個 y dataframe 以便後續合併資料

# 🛠️ 載入訓練好的CNN模型

In [ ]:
# 路徑設定
PROJECT_PATH = "/content/drive/MyDrive/Colab Notebooks/電腦視覺之深度學習/final_project"

# --- 載入CNN模型 ---
# CNN 模型路徑 (來自CNN_Draft輸出的CNN_model)
CNN_output_dir = os.path.join(PROJECT_PATH, "CNN_model")

model_path = os.path.join(CNN_output_dir, "2_cnn20d_final_lr1e-4_y3.pth")
device = torch.device('cuda')
model = CNN20d().to(device)

print(f"\n載入 CNN 權重從: {model_path}")
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval() # *** 務必設為評估模式 ***
print("✅ CNN 模型載入完成並設為 eval() 模式。")


載入 CNN 權重從: /content/drive/MyDrive/Colab Notebooks/電腦視覺之深度學習/final_project/CNN_model/2_cnn20d_final_lr1e-4_y3.pth
✅ CNN 模型載入完成並設為 eval() 模式。


In [ ]:
# --- 載入資料 ---
# 資料路徑 (Data_OHLC):
_subfolder_o = os.path.join(PROJECT_PATH, "Data_OHLC")

print(f"從 {_subfolder_o} 載入資料...")
# X_test
X_test = np.load(os.path.join(_subfolder_o, 'X_test.npy'))

# CNN y
y_test_0 = pd.read_csv(os.path.join(_subfolder_o, 'y_test_0.csv'))
y_test_1 = pd.read_csv(os.path.join(_subfolder_o, 'y_test_1.csv'))
y_test_2 = pd.read_csv(os.path.join(_subfolder_o, 'y_test_2.csv'))
y_test_3 = pd.read_csv(os.path.join(_subfolder_o, 'y_test_3.csv'))
y_test_4 = pd.read_csv(os.path.join(_subfolder_o, 'y_test_4.csv'))

print("✅ 資料載入完成。")

print(f"X_test shape: {X_test.shape}")
print(f"y_test_0 shape: {y_test_0.shape}")
print(f"y_test_1 shape: {y_test_1.shape}")
print(f"y_test_2 shape: {y_test_2.shape}")
print(f"y_test_3 shape: {y_test_3.shape}")
print(f"y_test_4 shape: {y_test_4.shape}")

從 /content/drive/MyDrive/Colab Notebooks/電腦視覺之深度學習/final_project/Data_OHLC 載入資料...
✅ 資料載入完成。
X_test shape: (5, 734, 28, 64, 60)
y_test_0 shape: (20552, 3)
y_test_1 shape: (20552, 3)
y_test_2 shape: (20552, 3)
y_test_3 shape: (20552, 3)
y_test_4 shape: (20552, 3)


# 🛠️ 進行推論


**Mode="top_k" or "threshold"**

In [ ]:
model.eval()
BATCH_SIZE = 64

y_test_list = [y_test_0, y_test_1, y_test_2, y_test_3, y_test_4]
final_matrix_list = [] # 用來存 5 個 (28xT) 的矩陣

print("\n=== 開始推論與維度轉換 ===")
# Mode="top_k"
Mode="threshold"
for i in range(5):
    # --- A. 準備 X ---
    # X_test[i] shape: (734, 28, 64, 60) -> Flatten -> (20552, 64, 60)
    # 這裡不用 unsqueeze(1) 因為你的模型 forward 第一行會自己做
    flattened_X = X_test[i].reshape(-1, 64, 60)
    tensor_X = torch.FloatTensor(flattened_X)

    # --- B. 準備 y (使用你的函數) ---
    current_y_df = y_test_list[i]
    # 呼叫 make_y_labels
    dataset_for_model, df_with_info = make_y_labels(current_y_df, mode=Mode)
    # 從 TensorDataset 取出 tensor
    tensor_y = dataset_for_model.tensors[0]

    # --- C. DataLoader ---
    dataset = TensorDataset(tensor_X, tensor_y)
    loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

    # --- D. 推論 (Inference) ---
    all_probs = [] # 存預測機率 (0~1)
    all_preds = [] # 存預測類別 (0/1)

    with torch.no_grad():
        for b_x, b_y in tqdm(loader, desc=f"Set {i}"):
            b_x = b_x.to(device)
            outputs = model(b_x)

            # outputs shape: (Batch, 2) -> [P(跌), P(漲)]
            # 取 index 1 代表看好的機率
            probs = outputs[:, 1]
            preds = torch.argmax(outputs, dim=1)

            all_probs.extend(probs.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    # 計算準確度 (Optional)
    acc = accuracy_score(tensor_y.numpy(), all_preds)
    print(f"Set {i} Accuracy : {acc:.4f}")

    # --- E. 轉換維度至 28 * T ---
    # 我們通常關心「模型預測的機率」，因為這決定了我們要買哪幾支
    # 總筆數 20552, 股票數 28 -> 天數 = 20552 / 28 = 734
    days = 734
    stocks = 28

    # 1. 先轉成 (天數, 股票數) -> (734, 28)
    # 邏輯：資料原本是 Day1[28筆] -> Day2[28筆]...
    prob_matrix_T_28 = np.array(all_probs).reshape(days, stocks)

    # 2. 轉置成 (股票數, 天數) -> (28, 734)
    prob_matrix_28_T = prob_matrix_T_28.T

    final_matrix_list.append(prob_matrix_28_T)
    print(f"Set {i} 輸出矩陣形狀: {prob_matrix_28_T.shape}")
print("\n=== 處理完成 ===")


=== 開始推論與維度轉換 ===


Set 0: 100%|██████████| 322/322 [00:46<00:00,  6.94it/s]


Set 0 Accuracy : 0.5030
Set 0 輸出矩陣形狀: (28, 734)


Set 1: 100%|██████████| 322/322 [00:45<00:00,  7.04it/s]


Set 1 Accuracy : 0.5036
Set 1 輸出矩陣形狀: (28, 734)


...

Set 2 Accuracy : 0.5023
Set 2 輸出矩陣形狀: (28, 734)


Set 3: 100%|██████████| 322/322 [00:45<00:00,  7.02it/s]


Set 3 Accuracy : 0.5014
Set 3 輸出矩陣形狀: (28, 734)


Set 4: 100%|██████████| 322/322 [00:45<00:00,  7.03it/s]

Set 4 Accuracy : 0.5036
Set 4 輸出矩陣形狀: (28, 734)

=== 處理完成 ===


In [ ]:
# 第0組，28支股票，在第0天的機率
print(final_matrix_list[0][:,0])

[9.9662530e-01 6.0956785e-03 9.9812299e-01 8.4836207e-02 7.0229924e-09
 3.3418366e-04 9.9952602e-01 9.9989378e-01 9.9851316e-01 2.1079030e-02
 9.9999750e-01 8.8029772e-01 1.0000000e+00 6.6929375e-18 4.4953663e-09
 9.9288321e-01 9.3628854e-01 5.3269707e-07 9.9999988e-01 3.2685800e-03
 5.9017460e-03 6.6281966e-11 9.9998534e-01 6.8579107e-09 9.9994993e-01
 1.2388439e-06 9.9999976e-01 1.0000000e+00]


# 🛠️ 儲存檔案

In [ ]:
# 儲存檔案
PROJECT_PATH = "/content/drive/MyDrive/Colab Notebooks/電腦視覺之深度學習/final_project"

CNN_testset_output = os.path.join(PROJECT_PATH, "CNN_testset_output")
os.makedirs(CNN_testset_output, exist_ok=True)

# 1. 轉換成一個大的 Numpy Array
# 形狀會從 List 變成 (5, 28, 734) 的 Array
all_predictions = np.array(final_matrix_list)

# 2. 設定檔名與儲存
npy_path = os.path.join(CNN_testset_output, "2_CNN_output_lr1e-4_y3.npy")
np.save(npy_path, all_predictions)

print(f"✅ 已儲存為 .npy 檔: {npy_path}")
print(f"儲存後的形狀: {all_predictions.shape}")

✅ 已儲存為 .npy 檔: /content/drive/MyDrive/Colab Notebooks/電腦視覺之深度學習/final_project/CNN_testset_output/2_CNN_output_lr1e-4_y3.npy
儲存後的形狀: (5, 28, 734)


In [ ]:
print(f"\n載入 CNN 權重從: {model_path}")
print(f"\n使用Mode : {Mode}")
print(f"✅ 已儲存為 .npy 檔: {npy_path}")


載入 CNN 權重從: /content/drive/MyDrive/Colab Notebooks/電腦視覺之深度學習/final_project/CNN_model/2_cnn20d_final_lr1e-4_y3.pth

使用Mode : threshold
✅ 已儲存為 .npy 檔: /content/drive/MyDrive/Colab Notebooks/電腦視覺之深度學習/final_project/CNN_testset_output/2_CNN_output_lr1e-4_y3.npy
